In [12]:
%pip install cmem-cmempy llama-index python-dotenv llama-index-vector-stores-postgres

Note: you may need to restart the kernel to use updated packages.


In [1]:
%load_ext dotenv
%dotenv
%reload_ext dotenv

In [14]:
import os

import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [15]:
!docker pull ankane/pgvector
!docker run -d -e POSTGRES_USER=eccenca -e POSTGRES_PASSWORD=eccenca -e POSTGRES_DB=eccenca --name pgvector-eccenca -p 5432:5432 ankane/pgvector

Using default tag: latest
latest: Pulling from ankane/pgvector
Digest: sha256:956744bd14e9cbdf639c61c2a2a7c7c2c48a9c8cdd42f7de4ac034f4e96b90f8
Status: Image is up to date for ankane/pgvector:latest
docker.io/ankane/pgvector:latest
docker: Error response from daemon: Conflict. The container name "/pgvector-eccenca" is already in use by container "96a9465720137f2fd7cc29e12a3fb68a7a18a782ff10af02498e5459d2fc8424". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [5]:
from IPython.display import Markdown
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database="eccenca",
    host="localhost",
    password="eccenca",
    port="5432",
    user="eccenca",
    table_name="prod-inst",
    embed_dim=1536,  # openai embedding dimension
    hnsw_kwargs={
        "hnsw_m": 16,
        "hnsw_ef_construction": 64,
        "hnsw_ef_search": 40,
        "hnsw_dist_method": "vector_cosine_ops",
    },
)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
query_engine = index.as_query_engine(similarity_top_k=30)
question = """What can you tell me about Dietlinde Boehme? \
Give me as many information and details as possible. \
Format the result nicely as Markdown. \
If sensible use list-style format. \
Also add a textual summary."""
answer = query_engine.query(question)

display(Markdown(f"### Question: _{question}_"))
display(Markdown(f"### Answer:\n\n{answer}"))
display(Markdown("#### Source nodes:"))
for node in answer.source_nodes:
    display(Markdown(f"> __Text:__ {node.text}\n>\n> __Score:__ {node.score}"))

PG Setup: Error creating HNSW index: (psycopg2.errors.SyntaxError) syntax error at or near "-"
LINE 1: CREATE INDEX IF NOT EXISTS data_prod-inst_embedding_idx ON p...
                                            ^

[SQL: CREATE INDEX IF NOT EXISTS data_prod-inst_embedding_idx ON public.data_prod-inst USING hnsw (embedding vector_cosine_ops) WITH (m = 16, ef_construction = 64)]
(Background on this error at: https://sqlalche.me/e/20/f405)


### Question: _What can you tell me about Dietlinde Boehme? Give me as many information and details as possible. Format the result nicely as Markdown. If sensible use list-style format. Also add a textual summary._

### Answer:

- **Name:** Dietlinde Boehme
- **Email:** Dietlinde.Boehme@company.org
- **Member of:** Marketing
- **Address:** Biebricher Allee 880, 65826 Neukalen
- **Phone Number:** (06582) 6550636
- **Managers:** Liese Adam, Valda Everhart, Lambert Faust, Yanka Schreiber, Baldwin Dirksen, Bert Blumstein, Henny Foth, Kevin Feigenbaum, Sabrina Geiger
- **Product Manager for:** 
  - V881-4674578 - Planck Dipole Driver Crystal
  - J555-1586043 - Transformer Inductor Encoder
  - O300-8464663 - Crystal Strain

**Summary:** Dietlinde Boehme is a member of the Marketing department. She can be reached at Dietlinde.Boehme@company.org or (06582) 6550636. Her address is Biebricher Allee 880, 65826 Neukalen. Dietlinde manages a number of individuals and is the product manager for several products including Planck Dipole Driver Crystal, Transformer Inductor Encoder, and Crystal Strain.

#### Source nodes:

> __Text:__ Dietlinde Boehme name Dietlinde Boehme
>
> __Score:__ 0.8683431305476333

...

> __Text:__ V881-4674578 - Planck Dipole Driver Crystal has product manager Dietlinde Boehme
>
> __Score:__ 0.77785780363854

> __Text:__ Arendt Beitel address text Dessauer Ufer 116, 08763 Olpe
>
> __Score:__ 0.7773857647452915

> __Text:__ J555-1586043 - Transformer Inductor Encoder has product manager Dietlinde Boehme
>
> __Score:__ 0.7766423757928906

> __Text:__ O300-8464663 - Crystal Strain has product manager Dietlinde Boehme
>
> __Score:__ 0.7764093875885231

> __Text:__ Dieterich Blau area of expertise Breaker
>
> __Score:__ 0.7747626245587768